In [1]:
import pandas as pd
import json
import os
import mapply
from google.cloud import bigquery
import gcsfs
import ast
import time
from tqdm import tqdm
client = bigquery.Client()
from vertexai.preview.language_models import TextGenerationModel
from smart_open import smart_open
from sqlalchemy import *
from sqlalchemy.engine import create_engine
mapply.init(
    n_workers=-1,
    chunk_size=1000,
    max_chunks_per_worker=0,
    progressbar=True
)
tqdm.pandas(desc='progress bar')
pd.set_option('display.max_colwidth', None)  # Set maximum width to terminal width
pd.set_option('display.expand_frame_repr', True)  # Expand DataFrame representation


/home/user/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


!pip install google-cloud-aiplatform

In [4]:
engine = create_engine('bigquery://us-gcp-ame-con-52dbb-sbx-1')

/tmp/ipykernel_1690/111781775.py:1: SADeprecationWarning: The dbapi() classmethod on dialect classes has been renamed to import_dbapi().  Implement an import_dbapi() classmethod directly on class <class 'pybigquery.sqlalchemy_bigquery.BigQueryDialect'> to remove this warning; the old .dbapi() classmethod may be maintained for backwards compatibility.
  engine = create_engine('bigquery://us-gcp-ame-con-52dbb-sbx-1')


In [5]:
data_frame = pd.read_sql_query(f"SELECT * FROM data_quality.meta_Prime_Pantry ;",engine)
data_frame.shape

/home/user/.local/lib/python3.10/site-packages/google/cloud/bigquery/client.py:572: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


(10813, 14)

In [ ]:
data_frame['title'].head(10)

In [6]:
data_frame['prompt'] = data_frame.mapply(lambda x: f"""You are an assistant that extracts product weight, count per pack, and total count details from product titles. When this information does not exist in the title, you should respond with "None". I will provide some examples for you to understand the task:

Title: V8 +Energy, Juice Drink with Green Tea, Pomegranate Blueberry, 8 oz. Can (4 packs of 6, Total of 24)
resonse: 8 oz. Can (4 packs of 6, Total of 24)

Title: ACE Night Wrist Sleep Support
resonse: None

Title: EO Botanical Sulfate Free Conditioner with Keratin, Coconut and Hibiscus, 8.4 oz
resonse: 8.4 oz

Title: EO Pure Performance Botanical Shampoo, Sulfate Free with Keratin for Chemically Treated Hair, Coconut &amp; Hibiscus, 8.4 Ounce
response: 8.4 Ounce

Title: Orbit Peppermint Sugarfree Gum, 3 Pack
response: 3 Pack

now its your turn

Title: {x['title']}
response:""",axis=1)

/home/user/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
100%|██████████| 10/10 [00:00<00:00, 13.18it/s]


#### function to perform  text bison LLM online predictions

In [7]:
#Loads the pre-trained text generation model 'text-bison' for subsequent text generation
text_model = TextGenerationModel.from_pretrained("text-bison@001")

In [3]:
client = bigquery.Client()

table_id = "us-gcp-ame-con-52dbb-sbx-1.data_quality.Prime_Pantry_attribute_extraction"
schema = [
    bigquery.SchemaField("asin", "STRING"),
    bigquery.SchemaField("attribute_extraction", "STRING"),
]

table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)

In [9]:

# Iterate over records and write them to the table
for idx, row in tqdm(data_frame.iterrows(), total=data_frame.shape[0]):
    response = text_model.predict(row['prompt'], temperature = 0.0, top_k = 1).text
    client.insert_rows(table, [[row['asin'], response]])
    time.sleep(1)


 71%|███████   | 7678/10813 [2:50:37<1:09:39,  1.33s/it]


KeyboardInterrupt: 

In [ ]:
def extract_product_attributes(prompt):
    text_model = TextGenerationModel.from_pretrained("text-bison@001")
    response = text_model.predict(prompt=prompt, temperature = 0.0, top_k = 1).text
    time.sleep(1)
    return response

In [ ]:
# samples to test results
sample_data = data_frame.sample(n=200, random_state=42).reset_index(drop=True)

In [ ]:

#fu
sample_data['attribute_extraction'] = sample_data['prompt'].progress_apply(lambda x: extract_product_attributes(x))


In [ ]:
sample_data[['title','attribute_extraction']].to_csv("Attribute extraction.csv",index=False)